In [1]:
import os
import openai
import tempfile
import requests
from IPython.display import Audio, clear_output
from elevenlabs import generate, play, set_api_key, voices, Models

openai_api_key = os.environ["OPENAI_API_KEY"]
eleven_api_key = os.environ["ELEVEN_API_KEY"]

# Configuro GPT-4 Y Text-to-Speech API
openai.api_key = openai_api_key
set_api_key(eleven_api_key)

voice_list = voices()

Se selecciona la voy a utilizar

In [8]:
import ipywidgets as widgets

voice_labels = [voice.category + " voice: " + voice.name for voice in voice_list]

voice_id_dropdown = widgets.Dropdown(
    options=voice_labels,
    value=voice_labels[0],
    description="Seleccione una voz:",
)

display(voice_id_dropdown)

Dropdown(description='Seleccione una voz:', options=('premade voice: Rachel', 'premade voice: Domi', 'premade …

Digo la pregunta que tiene que resolver GPT

In [5]:
# grabar el audio con el microfono con pyaudio de la computadora para usarlo en OpenAI
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
FILE_PATH = "audios/output.wav"

def record_audio(file_path, format, channels, rate, chunk, record_seconds):
    p = pyaudio.PyAudio()

    stream = p.open(format=format, 
                    channels=channels, 
                    rate=rate, 
                    input=True, 
                    frames_per_buffer=chunk)

    print("Comenzando la grabación...")

    frames = []
    for i in range(0, int(rate / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)

    print("Fin de la grabación.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(file_path, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(rate)
    wf.writeframes(b"".join(frames))
    wf.close()

def transcribe_audio(file_path):
    if os.path.exists(file_path):
        audio_file = open(file_path, "rb")
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
        return transcript
    else:
        print("El archivo no existe")
        return None

def obtener_texto_de_microfono():
    record_seconds = 4  # puedes cambiar este valor a la duración deseada
    record_audio(FILE_PATH, FORMAT, CHANNELS, RATE, CHUNK, record_seconds)
    transcript = transcribe_audio(FILE_PATH)
    if transcript:
        print(transcript)
    return transcript

Configuración e interacción con ChatGPT

In [7]:
from mutagen.mp3 import MP3
import IPython.display as ipd
import time

chatgpt_model = "gpt-3.5-turbo"
chatgpt_system = "Eres asistente muy util para conversar. Tus respuestas no deben ser largas."

# Encuentra el índice de la opción seleccionada
selected_voice_index = voice_labels.index(voice_id_dropdown.value)
selected_voice_id    = voice_list[selected_voice_index].voice_id

# Function para obtener la respuesta GPT-4
def get_gpt_response(prompt):
    response = openai.ChatCompletion.create(
        model=chatgpt_model,
        messages=[
            {"role": "system", "content": chatgpt_system},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Función principal para interactuar con GPT
def interact_with_gpt(prompt):
    response_text = get_gpt_response(prompt)

    import requests

    CHUNK_SIZE = 1024
    url = "https://api.elevenlabs.io/v1/text-to-speech/" + selected_voice_id

    headers = {
      "Accept": "audio/mpeg",
      "Content-Type": "application/json",
      "xi-api-key": eleven_api_key
    }

    data = {
      "text": response_text,
      "model_id" : "eleven_multilingual_v1",
      "voice_settings": {
        "stability": 0.4,
        "similarity_boost": 1.0
      }
    }

    response = requests.post(url, json=data, headers=headers)
    
    # Salva el audio en un archivo temporal
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
            if chunk:
                f.write(chunk)
        f.flush()
        temp_filename = f.name

    return temp_filename

#response_audio_file = interact_with_gpt(transcript.text)
# autoplay audio
#play(response_audio_file, notebook=True)

# Función para interactuar continuamente con GPT
def continuous_interaction():
    while True:
        clear_output(wait=True)
        prompt = obtener_texto_de_microfono()
        if 'salir' in prompt.text.lower():
            break
        audio_file = interact_with_gpt(prompt.text)
        # Calcula la duración del audio
        audio = MP3(audio_file)
        duration = audio.info.length
        # Reproduce el audio
        ipd.display(ipd.Audio(audio_file, autoplay=True))
        # Pausa la ejecución del programa por la duración del audio
        time.sleep(duration + 5)

# Example usage
continuous_interaction()

Comenzando la grabación...
Fin de la grabación.
{
  "text": "Salir del programa."
}
